In [1]:
# permite a utilização de caminhos relativos ao projeto, mesmo no diretório de notebooks

from knapsax.utils import setrootdir

setrootdir("knapsax")

'Directory knapsax successfully loaded as current working directory.'

In [2]:
from typing import List, Tuple, Dict, Callable
import random
import copy

import numpy as np

from knapsax.optimization import Problem, Maximize, Solution, Knapsack

In [3]:
knapsack = Knapsack(instance_file="data/knapsack-instance.txt")
knapsack

Knapsack(file=data/knapsack-instance.txt, n_items=100, capacity=1550)

In [4]:
class HCSolution(Solution):
    def __init__(self, knapsack):
        super().__init__(knapsack)
        self.available_items = []

    def generate_solution(self):
        self.available_items = list(range(self.knapsack.n_items))

        random.shuffle(self.available_items)

        while self.available_items:
            item_index = self.available_items.pop(0)
            if not self.add_item(item_index):
                self.available_items.append(item_index)
                break

            random.shuffle(self.available_items)

    def move(self):
        if not self.items:
            return

        index_out = random.randint(0, len(self.items) - 1)
        item_out = self.items.pop(index_out)
        self.total_weight -= self.knapsack.items[item_out].weight
        self.total_value -= self.knapsack.items[item_out].value

        self.available_items.append(item_out)
        random.shuffle(self.available_items)

        while self.available_items:
            item_index = self.available_items[0]
            if self.total_weight + self.knapsack.items[item_index].weight <= self.knapsack.capacity:
                self.available_items.pop(0)
                self.add_item(item_index)
                random.shuffle(self.available_items)
            else:
                self.available_items.append(item_index)
                break

In [5]:
class HillClimbing:
    def __init__(self, problem, knapsack, max_iterations=1000):
        self.problem = problem
        self.knapsack = knapsack
        self.max_iterations = max_iterations
        self.n_iterations = 0
        self._called = False

        self.initial_solution = HCSolution(knapsack)
        self.initial_solution.generate_solution()

        self.best_solution = copy.deepcopy(self.initial_solution)
        self.best_cost = self.best_solution.total_value

        self.history = {
            "best_solution": [],
            "best_cost": [],
        }

    def run(self):
        if self._called:
            raise RuntimeError("This method can only be called once.")

        current_solution = copy.deepcopy(self.initial_solution)

        while self.n_iterations < self.max_iterations:
            current_solution.move()
            new_cost = current_solution.total_value

            if self.problem(new_cost, self.best_cost):
                self.best_solution = copy.deepcopy(current_solution)
                self.best_cost = new_cost

            self.history["best_solution"].append(copy.deepcopy(self.best_solution))
            self.history["best_cost"].append(self.best_cost)

            self.n_iterations += 1

        self._called = True

    def __str__(self):
        return (f"Iteration: {self.n_iterations}\n\n"
                f"Best cost: {self.best_cost}\n\n"
                f"Best solution weight: {self.best_solution.total_weight}\n\n"
                f"Best solution: {self.best_solution.items}")

    def __repr__(self):
        return self.__str__()


In [6]:
hc = HillClimbing(
    problem=Maximize(),
    knapsack=knapsack,
    max_iterations=20_000,
)

hc.run()

hc

Iteration: 20000

Best cost: 2037

Best solution weight: 1541

Best solution: [33, 27, 87, 56, 74, 55, 6, 24, 51, 16, 15, 83, 35, 11, 26, 79, 40, 22, 14, 53, 94, 3, 97, 37, 95, 5, 63, 4, 75, 29, 20, 92, 57, 85, 60, 12, 13, 99, 68, 2]